In [1]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, make_scorer
import pandas as pd
import numpy as np
import joblib

# 1.AD vs NC

In [2]:
data_dir = '/Users/imdohyeon/Documents/PythonWorkspace/4n/preprocessSeg/extracted/ver4(edf4CSP)/features_nc_ad_csp.csv'
data = pd.read_csv(data_dir)

In [3]:
data

,kurtosis_Fp1,kurtosis_Fp2,kurtosis_F7,kurtosis_F3,kurtosis_Fz,kurtosis_F4,kurtosis_F8,kurtosis_T7,kurtosis_C3,kurtosis_Cz,...,spectral_entropy_Pz,spectral_entropy_P4,spectral_entropy_P8,spectral_entropy_O1,spectral_entropy_O2,csp0,csp1,csp2,csp3,label
0,-0.418358,0.332468,-0.342700,-0.434762,-0.363083,1.541473,3.803859,-0.366574,-0.361034,-0.317023,...,2.867881,2.897658,3.226472,3.215344,3.264944,-12.782493,-14.40648,-13.659794,-12.371691,1
1,-0.553936,0.645437,-0.499525,-0.413223,-0.233330,1.566611,3.691952,-0.446604,-0.195897,-0.095939,...,3.141803,3.162673,3.711791,3.533822,3.642025,-12.782493,-14.40648,-13.659794,-12.371691,1
2,1.028320,-0.381386,0.390925,0.612009,0.007393,-1.033033,-0.263930,-0.269617,-0.003700,-1.806611,...,4.035325,4.044881,4.029477,4.004199,4.097106,-12.782493,-14.40648,-13.659794,-12.371691,1
3,1.024712,-0.350840,0.469474,0.555023,-0.059696,-0.864880,0.112843,-0.260772,-0.060767,-1.878602,...,3.824710,3.850665,3.836143,3.776048,3.924873,-12.782493,-14.40648,-13.659794,-12.371691,1
4,-0.007232,0.071214,-0.244308,-0.243306,-0.211503,-0.158064,4.221788,-0.248971,-0.280709,-0.297559,...,2.997639,3.016220,3.331265,3.395650,3.406698,-12.782493,-14.40648,-13.659794,-12.371691,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4166,-0.400714,-0.344705,0.841186,3.154313,-0.749571,-0.234422,-0.933491,-0.815349,-0.923803,-0.869718,...,4.181929,4.305306,4.995574,4.345621,4.280138,-12.782493,-14.40648,-13.659794,-12.371691,0
4167,0.196473,0.205968,-0.230757,0.297017,-0.288704,-1.187940,-1.216336,0.340628,-1.160514,-1.294505,...,4.153778,4.289267,4.884699,4.500191,4.388379,-12.782493,-14.40648,-13.659794,-12.371691,0
4168,-0.159535,-0.032668,0.049289,0.301747,-1.062893,-0.308668,-1.548534,-0.540624,-0.500203,-1.501149,...,4.050663,4.196556,4.726267,4.345120,4.224140,-12.782493,-14.40648,-13.659794,-12.371691,0
4169,-0.689377,-0.455913,-0.331188,-0.298873,-1.089699,-0.470368,-0.442167,-0.021996,-0.644215,-0.376904,...,5.172031,4.664239,4.391314,4.659940,4.639465,-12.782493,-14.40648,-13.659794,-12.371691,0


- Multi class에서 L1 정규화 적용 시 모든 주파수 도메인 피처를 제거하였음. 주파수 도메인 피처보다 다른 피처가 더 중요하다고 판단되어, 주파수 도메인 피처는 추출하지 않고 진행. 

In [4]:
# 데이터 셔플
data = data.sample(frac=1).reset_index(drop=True)
data

,kurtosis_Fp1,kurtosis_Fp2,kurtosis_F7,kurtosis_F3,kurtosis_Fz,kurtosis_F4,kurtosis_F8,kurtosis_T7,kurtosis_C3,kurtosis_Cz,...,spectral_entropy_Pz,spectral_entropy_P4,spectral_entropy_P8,spectral_entropy_O1,spectral_entropy_O2,csp0,csp1,csp2,csp3,label
0,0.094166,0.243561,-0.067493,1.097593,-1.083213,-0.026925,0.770459,-0.684447,-1.735126,-1.635081,...,3.524055,3.429300,3.578766,3.593731,3.664399,-12.782493,-14.406480,-13.659794,-12.371691,0
1,1.048310,1.561793,0.732381,-0.020833,1.356178,1.502923,1.138397,0.044538,-0.627912,0.139055,...,3.025852,3.083477,3.273435,3.413714,3.420737,-12.782493,-14.406480,-13.659794,-12.371691,0
2,0.234690,0.010824,0.182618,-1.189777,-1.045144,-0.669141,-0.770656,-0.377408,-0.674622,-1.039688,...,4.259545,3.983663,4.175541,4.303302,4.321071,-12.782493,-14.406480,-13.659794,-12.371691,0
3,-0.442908,-0.321346,-0.072194,-0.666167,-0.700846,-0.779658,-0.182274,-0.453784,0.129934,0.274705,...,5.070996,4.815733,5.031536,4.852264,5.014741,-12.782493,-14.406480,-13.659794,-12.371691,0
4,-2.891578,0.182022,0.000372,-0.902028,0.420798,1.273699,1.536816,0.656844,0.869373,0.462113,...,3.845521,3.735827,3.703877,3.689897,3.738565,-12.782493,-14.406480,-13.659794,-12.371691,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4166,-0.209736,-0.259131,-0.204059,-0.190726,-0.218989,-0.262707,-0.273758,-0.205366,-0.213527,-0.234958,...,2.692118,2.805943,2.840444,2.955232,2.867791,-12.782493,-14.406480,-13.659794,-12.371691,1
4167,0.107128,0.670481,0.681720,0.171906,0.039903,0.676687,0.310891,-0.226676,0.236700,1.585693,...,3.062089,3.151496,3.324375,3.207429,3.270696,-12.782493,-14.406480,-13.659794,-12.371691,1
4168,0.454345,-0.297104,-0.123153,0.225835,-0.300206,-0.876409,-1.580045,-1.000676,1.302775,-1.416208,...,2.665044,2.603924,2.647319,3.005577,3.346210,-12.782493,-14.406480,-13.659794,-12.371691,1
4169,-1.413828,-0.274010,1.228499,2.055227,1.845025,-0.260628,0.007553,-0.373277,-0.568715,1.245819,...,3.520749,3.568256,5.374557,5.352531,5.350140,-12.782493,-14.406480,-13.659794,-12.371691,0


In [5]:
# 데이터 나누기
X = data.drop(['label'], axis=1)
y = data['label']
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 1-1. L1 Regularization + RF (Grid)

In [6]:
# Lasso를 사용한 특징 선택
lasso = Lasso(max_iter=10000)

# Random Forest 모델
rf_model = RandomForestClassifier(random_state=42)

# 파이프라인 구축
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(estimator=lasso)),
    ('random_forest', rf_model)
])

# 하이퍼파라미터 그리드 정의
param_grid = {
    'feature_selection__estimator__alpha': [0.001, 0.01, 0.1, 1],  # Lasso의 alpha 값
    'random_forest__n_estimators': [50, 100, 200],  # Random Forest의 트리 개수
    'random_forest__max_depth': [None, 10, 20, 30]  # Random Forest의 최대 깊이
}
# Weighted F1 스코어 사용
f1_scorer = make_scorer(f1_score, average='weighted')

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=f1_scorer, n_jobs=-1, verbose=1)

In [7]:
# 모델 학습
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py", line 328, in fit
    X, y

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_selection',
                                        SelectFromModel(estimator=Lasso(max_iter=10000))),
                                       ('random_forest',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'feature_selection__estimator__alpha': [0.001, 0.01,
                                                                 0.1, 1],
                         'random_forest__max_depth': [None, 10, 20, 30],
                         'random_forest__n_estimators': [50, 100, 200]},
             scoring=make_scorer(f1_score, average=weighted), verbose=1)

In [8]:
# 최적 파라미터 및 성능 출력
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score:", grid_search.best_score_)

# 최적 모델 저장
best_model_rf = grid_search.best_estimator_
joblib.dump(best_model_rf, "/Users/imdohyeon/Documents/PythonWorkspace/4n/model/best_l1rf_model_ver4_csp.pkl")
print("Model saved to 'best_l1rf_model_ver4.pkl'")

# 테스트 세트 평가
y_pred = grid_search.best_estimator_.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Test Accuracy: {accuracy:.2f}", f"Test F1: {f1:.2f}")

Best Parameters: {'feature_selection__estimator__alpha': 0.001, 'random_forest__max_depth': None, 'random_forest__n_estimators': 200}
Best CV Score: 0.8829827932865534
Model saved to 'best_l1rf_model_ver4.pkl'
Test Accuracy: 0.90 Test F1: 0.90


## 1-2. L1 Regularization + XGB(Grid) + Asymmetric Loss

In [9]:
import xgboost as xgb
from xgboost import XGBClassifier

In [10]:
# Lasso를 사용한 특징 선택
lasso = Lasso(max_iter=10000)

# XGBoost 모델
xgb_model = XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    eval_metric='logloss'
)

# 파이프라인
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(estimator=lasso)),
    ('xgboost', xgb_model)
])

# Grid Search 설정
param_grid = {
    'feature_selection__estimator__alpha': [0.001, 0.01, 0.1],
    'xgboost__n_estimators': [50, 100, 200],
    'xgboost__max_depth': [3, 5, 7],
    'xgboost__learning_rate': [0.01, 0.1, 0.3]
}

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_score': make_scorer(f1_score, average='weighted')
}

grid_search_xgb = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring=scoring,
    refit='accuracy',
    verbose=1,
    n_jobs=-1
)

In [11]:
# 모델 학습
grid_search_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_selection',
                                        SelectFromModel(estimator=Lasso(max_iter=10000))),
                                       ('xgboost',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric='logloss',
                                                      gamma=None, gpu_id=None,
                   

In [12]:
# 최적 파라미터 출력
print("Best Parameters:", grid_search_xgb.best_params_)

# 테스트 데이터 평가
y_pred = grid_search_xgb.best_estimator_.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"Test F1-Score: {f1_score(y_test, y_pred, average='weighted'):.2f}")

Best Parameters: {'feature_selection__estimator__alpha': 0.001, 'xgboost__learning_rate': 0.1, 'xgboost__max_depth': 7, 'xgboost__n_estimators': 200}
Test Accuracy: 0.92
Test F1-Score: 0.92


In [13]:
# 최적 모델 저장
joblib.dump(grid_search_xgb.best_estimator_, "/Users/imdohyeon/Documents/PythonWorkspace/4n/model/best_l1xgb_logloss_model_ver4.pkl")
print("Model saved to 'best_l1xgb_logloss_model_ver4.pkl'")

Model saved to 'best_l1xgb_logloss_model_ver4.pkl'


# 2. MCI vs NC

In [14]:
data_mci_dir = '/Users/imdohyeon/Documents/PythonWorkspace/4n/preprocessSeg/extracted/ver4(edf4CSP)/features_nc_ad_csp.csv'
data_mci = pd.read_csv(data_mci_dir)

In [15]:
data_mci = data_mci.sample(frac=1).reset_index(drop=True)
# 데이터 나누기
X_mci = data_mci.drop(['label'], axis=1)
y_mci = data_mci['label']
# 데이터 분할
X_train_mci, X_test_mci, y_train_mci, y_test_mci = train_test_split(X_mci, y_mci, test_size=0.3, random_state=42)

In [16]:
# Lasso를 사용한 특징 선택
lasso = Lasso(max_iter=10000)

## 2-1. RF

In [17]:
# Random Forest 모델
rf_model = RandomForestClassifier(random_state=42)

# 파이프라인 구축
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(estimator=lasso)),
    ('random_forest', rf_model)
])

# 하이퍼파라미터 그리드 정의
param_grid = {
    'feature_selection__estimator__alpha': [0.001, 0.01, 0.1, 1],  # Lasso의 alpha 값
    'random_forest__n_estimators': [50, 100, 200],  # Random Forest의 트리 개수
    'random_forest__max_depth': [None, 10, 20, 30]  # Random Forest의 최대 깊이
}
# Weighted F1 스코어 사용
f1_scorer = make_scorer(f1_score, average='weighted')

grid_search_rf_mci = GridSearchCV(pipeline, param_grid, cv=5, scoring=f1_scorer, n_jobs=-1, verbose=1)

In [18]:
# 모델 학습
grid_search_rf_mci.fit(X_train_mci, y_train_mci)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py", line 328, in fit
    X, y

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_selection',
                                        SelectFromModel(estimator=Lasso(max_iter=10000))),
                                       ('random_forest',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'feature_selection__estimator__alpha': [0.001, 0.01,
                                                                 0.1, 1],
                         'random_forest__max_depth': [None, 10, 20, 30],
                         'random_forest__n_estimators': [50, 100, 200]},
             scoring=make_scorer(f1_score, average=weighted), verbose=1)

In [19]:
# 최적 파라미터 및 성능 출력
print("Best Parameters:", grid_search_rf_mci.best_params_)
print("Best CV Score:", grid_search_rf_mci.best_score_)

# 최적 모델 저장
best_model_rf_mci = grid_search_rf_mci.best_estimator_
joblib.dump(best_model_rf_mci, "/Users/imdohyeon/Documents/PythonWorkspace/4n/model/best_l1rf_model_ver4_csp_mci.pkl")
print("Model saved to 'best_l1rf_model_ver4_csp_mci.pkl'")

# 테스트 세트 평가
y_pred_mci = grid_search_rf_mci.best_estimator_.predict(X_test_mci)
accuracy = accuracy_score(y_test_mci, y_pred_mci)
f1 = f1_score(y_test_mci, y_pred_mci, average='weighted')
print(f"Test Accuracy: {accuracy:.2f}", f"Test F1: {f1:.2f}")

Best Parameters: {'feature_selection__estimator__alpha': 0.001, 'random_forest__max_depth': 20, 'random_forest__n_estimators': 200}
Best CV Score: 0.8905902350038606
Model saved to 'best_l1rf_model_ver4_csp_mci.pkl'
Test Accuracy: 0.90 Test F1: 0.90


## 2-2.XGB

In [20]:
# XGBoost 모델
xgb_model = XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    eval_metric='logloss'
)

# 파이프라인
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(estimator=lasso)),
    ('xgboost', xgb_model)
])

# Grid Search 설정
param_grid = {
    'feature_selection__estimator__alpha': [0.001, 0.01, 0.1],
    'xgboost__n_estimators': [50, 100, 200],
    'xgboost__max_depth': [3, 5, 7],
    'xgboost__learning_rate': [0.01, 0.1, 0.3]
}

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_score': make_scorer(f1_score, average='weighted')
}

grid_search_xgb_mci = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring=scoring,
    refit='accuracy',
    verbose=1,
    n_jobs=-1
)

In [21]:
# 모델 학습
grid_search_xgb_mci.fit(X_train_mci, y_train_mci)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_selection',
                                        SelectFromModel(estimator=Lasso(max_iter=10000))),
                                       ('xgboost',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric='logloss',
                                                      gamma=None, gpu_id=None,
                   

In [22]:
# 최적 파라미터 출력
print("Best Parameters:", grid_search_xgb_mci.best_params_)

# 테스트 데이터 평가
y_pred_mci = grid_search_xgb_mci.best_estimator_.predict(X_test_mci)
print(f"Test Accuracy: {accuracy_score(y_test_mci, y_pred_mci):.2f}")
print(f"Test F1-Score: {f1_score(y_test_mci, y_pred_mci, average='weighted'):.2f}")

Best Parameters: {'feature_selection__estimator__alpha': 0.001, 'xgboost__learning_rate': 0.1, 'xgboost__max_depth': 7, 'xgboost__n_estimators': 200}
Test Accuracy: 0.92
Test F1-Score: 0.92


In [23]:
# 최적 모델 저장
joblib.dump(grid_search_xgb_mci.best_estimator_, "/Users/imdohyeon/Documents/PythonWorkspace/4n/model/best_l1xgb_logloss_model_ver4_mci.pkl")
print("Model saved to 'best_l1xgb_logloss_model_ver4_mci.pkl'")

Model saved to 'best_l1xgb_logloss_model_ver4_mci.pkl'


## 나머지 절반 NC 데이터 테스트

- CSP는 두 개 이상의 클래스가 존재하는 데이터에 대하여 처리 가능한 기법이기 때문에 NC2만 따로 전처리하는 것은 불가능